In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import pandasql as ps
from pandasql import sqldf

sns.set(font_scale=3)
sns.set(color_codes=True)
%matplotlib inline
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz,process



plt.rcParams["figure.figsize"] = [16, 16] 


pysqldf = lambda q: sqldf(q, globals())


In [ ]:
def replace_space_in_col_names(df):
    new_cols = [x.replace(" ","_") for x in df.columns]
    df.columns = new_cols

In [ ]:
hospitals_df = pd.read_csv("../web_data/Hospital_General_Information.csv")
sf_account_df = pd.read_excel("../data/QuVa-GAIL DataRequest_6_7_8_Addition2.xlsx","Account")
replace_space_in_col_names(hospitals_df)
replace_space_in_col_names(sf_account_df)
hospitals_df.City = hospitals_df.City.str.lower()
hospitals_df.ZIP_Code = hospitals_df.ZIP_Code.astype(str)
hospitals_df.Provider_ID = hospitals_df.Provider_ID.astype(str)
sf_account_df.BillingPostalCode = sf_account_df.BillingPostalCode.astype(str)
sf_account_df.ShippingCity = sf_account_df.ShippingCity.str.lower()



In [ ]:
hospitals_subset = hospitals_df[["Provider_ID","Hospital_Name","Address","ZIP_Code","City","State"]]
sf_account_subset = sf_account_df[["Account_id","Name","BillingCity","BillingState","BillingPostalCode","BillingStreet","ShippingPostalCode","ShippingCity"]]
sf_account_subset["ShippingPostalCode"] = sf_account_subset.ShippingPostalCode.apply(lambda x : x.split('-')[0] if "-"  in str(x) else str(x))
sf_account_subset.head()

In [ ]:
len(sf_account_subset[~sf_account_subset.ShippingPostalCode.isnull()])
temps = "69810"
temps[:-1]

sf_account_subset["PartialShippingPostalCode"] = sf_account_subset.ShippingPostalCode.apply(lambda x : x[:-1])
hospitals_subset["Partial_ZIP_Code"] = hospitals_subset.ZIP_Code.apply(lambda x : x[:-1])

In [ ]:
merged_by_zipcode = pd.merge(sf_account_subset,hospitals_subset,left_on="ShippingPostalCode",right_on="ZIP_Code",how='left')
matched_merged_by_zipcode = merged_by_zipcode[~merged_by_zipcode.City.isnull()]
matched_merged_by_zipcode["fuzzy_name_score"] = matched_merged_by_zipcode.apply(lambda x : fuzz.QRatio(x.Name.lower(),x.Hospital_Name.lower()),axis=1)
matched_merged_by_zipcode["fuzzy_name_rank"] = matched_merged_by_zipcode.groupby('Account_id')['fuzzy_name_score'].rank(ascending=False)
matched_merged_by_zipcode = matched_merged_by_zipcode[matched_merged_by_zipcode.fuzzy_name_rank == 1]
manual_merged_by_zipcode = matched_merged_by_zipcode[matched_merged_by_zipcode.fuzzy_name_score < 50]
matched_merged_by_zipcode = matched_merged_by_zipcode[matched_merged_by_zipcode.fuzzy_name_score >= 50]

non_matched_merged_by_zipcode = merged_by_zipcode[merged_by_zipcode.City.isnull()]
non_matched_merged_by_zipcode = pd.merge(non_matched_merged_by_zipcode,hospitals_subset,left_on='ShippingCity',right_on='City',how='left')
non_matched_merged_by_zipcode.fillna(' ')
non_matched_merged_by_zipcode.Hospital_Name_y = non_matched_merged_by_zipcode.Hospital_Name_y.astype(str)
non_matched_merged_by_zipcode["fuzzy_name_score"] = non_matched_merged_by_zipcode.apply(lambda x : fuzz.QRatio(x.Name.lower(),x.Hospital_Name_y.lower()),axis=1)
non_matched_merged_by_zipcode["fuzzy_name_rank"] = non_matched_merged_by_zipcode.groupby('Account_id')['fuzzy_name_score'].rank(ascending=False)
non_matched_merged_by_zipcode = non_matched_merged_by_zipcode[non_matched_merged_by_zipcode.fuzzy_name_rank==1]
manual_non_matched_merged = non_matched_merged_by_zipcode[non_matched_merged_by_zipcode.fuzzy_name_score < 50]
non_matched_merged_by_zipcode = non_matched_merged_by_zipcode[non_matched_merged_by_zipcode.fuzzy_name_score >= 50]


In [ ]:
manual_non_matched_merged = manual_non_matched_merged[manual_non_matched_merged.Hospital_Name_y !='nan']
manual_non_matched_merged["PartialZipMatch"] = manual_non_matched_merged.apply(lambda x : 'True' if abs(int(x.ZIP_Code_y[-2:]) - int( x.ShippingPostalCode[-2:])) == 1 and x.ZIP_Code_y[:-2] ==  x.ShippingPostalCode[:-2] else 'False', axis=1)
manual_non_matched_merged[["Account_id","Name","ShippingCity","Hospital_Name_y","City_y","fuzzy_name_score","ZIP_Code_y","ShippingPostalCode","PartialZipMatch"]].to_clipboard(index=False)
non_matched_merged_by_zipcode[["Account_id","Name","ShippingCity","BillingStreet","Hospital_Name_y","City_y","Address_y","fuzzy_name_score","ZIP_Code_y","ShippingPostalCode"]].to_clipboard(index=False)


In [ ]:
#matched_merged_by_zipcode[["Account_id","Name","ShippingCity","Hospital_Name","City","fuzzy_name_score","ZIP_Code","ShippingPostalCode"]].to_clipboard(index=False)
non_matched_merged_by_zipcode[["Account_id","Name","ShippingCity","Hospital_Name_y","City_y","fuzzy_name_score","Provider_ID_x"]].to_clipboard(index=False)


non_matched_merged_by_zipcode


manual_merged_by_zipcode.to_clipboard(index=False)
#non_matched_merged_by_zipcode['Fuzzy_Hospital_Name'] = non_matched_merged_by_zipcode.head(3).Name.apply(lambda x : process.extractOne(x,hospitals_name_address_list)[0])

In [ ]:
non_matched_merged_by_zipcode
matched_merged_by_zipcode.to_clipboard(index=False)

In [ ]:
#Grap the manual assignment data

manual_assignment_non_matched_merged_df = pd.read_excel("hospitals_sf_accounts_mappings.xlsx","manual_non_matched_merged")
#manual_assignment_non_matched_merged_df = manual_assignment_non_matched_merged_df[manual_assignment_non_matched_merged_df.match == 1]
#adding provider id 
manual_assignment_non_matched_merged_df = pd.merge(manual_assignment_non_matched_merged_df,hospitals_df[["Provider_ID","Hospital_Name","City"]],left_on=['Hospital_Name_y','City_y'],right_on=['Hospital_Name','City'])
manual_assignment_non_matched_merged_df = manual_assignment_non_matched_merged_df.drop(["Hospital_Name_y","City_y"],axis=1)
manual_assignment_non_matched_merged_df = manual_assignment_non_matched_merged_df.rename(columns={"ZIP_Code_y":"ZIP_Code"})


#non_matched_merged_by_zipcode Sheet
manual_assignment_non_matched_merged_by_zipcode_df = pd.read_excel("hospitals_sf_accounts_mappings.xlsx","non_matched_merged_by_zipcode")
#adding provider id 
manual_assignment_non_matched_merged_by_zipcode_merged_df = pd.merge(manual_assignment_non_matched_merged_by_zipcode_df,hospitals_df[["Provider_ID","Hospital_Name","City"]],left_on=['Hospital_Name_y','City_y'],right_on=['Hospital_Name','City'])
manual_assignment_non_matched_merged_by_zipcode_merged_df = manual_assignment_non_matched_merged_by_zipcode_merged_df.drop(["Hospital_Name_y","City_y"],axis=1)
manual_assignment_non_matched_merged_by_zipcode_merged_df = manual_assignment_non_matched_merged_by_zipcode_merged_df.rename(columns={"ZIP_Code_y":"ZIP_Code"})

#manual_merged_by_zipcode Sheet
manual_assignment_manual_merged_by_zipcode_df = pd.read_excel("hospitals_sf_accounts_mappings.xlsx","manual_merged_by_zipcode")


#subset cols 
subset_cols = ['Account_id','Name','ShippingPostalCode','ShippingCity','Provider_ID','Hospital_Name','ZIP_Code','City','match','fuzzy_name_score']
merged_manual_all_df = pd.concat([manual_assignment_non_matched_merged_df[subset_cols],manual_assignment_non_matched_merged_by_zipcode_merged_df[subset_cols],manual_assignment_manual_merged_by_zipcode_df[subset_cols]])
matched_merged_by_zipcode['match'] = 1
merged_all_df = pd.concat([matched_merged_by_zipcode[subset_cols],merged_manual_all_df])
#merged_all_df = merged_all_df[merged_all_df.match==1]
merged_all_df.to_csv('sf_account_provider_id_fuzzy_match_v1.csv',index=False)


In [ ]:
len(merged_all_df)/1130

In [ ]:
tmp1 = pd.merge(merged_all_df,sf_account_df[["Account_id","ShippingStreet"]],on="Account_id")
tmp2 = pd.merge(tmp1,hospitals_df[["Provider_ID","Address"]],on="Provider_ID",how="left")
tmp2["match"] = tmp2.match.apply(lambda x : 0 if x == '' else x)

tmp2.fillna(0).to_csv("sf_account_provider_id_fuzzy_match_with_address_v1.csv",index=False)

In [ ]:
v2 = pd.read_excel("../data/matching/Copy of sf_account_provider_id_fuzzy_match_with_address_v2.xlsx")

In [ ]:
non_matched= v2[v2.match == 0]
non_matched = non_matched[~non_matched.SF_Name.str.contains('OBSOLETE')]
non_matched.columns
non_matched = non_matched[["SF_Account_id","SF_ShippingStreet","SF_ShippingCity",'SF_ShippingPostalCode']]

sf_account_df = sf_account_df[sf_account_df["Customer_Prospect_Type__c"].isin(["Hospital","Hospital System"])]
sf_account_df = sf_account_df[~sf_account_df["Name"].str.contains("OBSOLETE")]


non_matched_joined = pd.merge(non_matched,sf_account_df[["Account_id","Name","BillingState"]],left_on="SF_Account_id",right_on="Account_id")
non_matched_joined = non_matched_joined.drop(["Account_id"],axis=1)


In [59]:
v2_with_match_flag = v2[["SF_Account_id","match","Third_Party_Hospital_Name","Third_Party_ZIP_Code","Third_Party_City","Third_Party_Address","Third_Party_Provider_ID"]]
v2_with_match_flags_with_sf = pd.merge (v2_with_match_flag,sf_account_df,left_on="SF_Account_id",right_on="Account_id",how='outer')



In [56]:
v2_with_match_flags_with_sf.to_csv("../data/matching/sf_all_accounts_with_manual_matching.csv",index=False)

In [57]:
len(sf_account_df)

1864